In [1]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import time
import math

# -[MPU6050 - Registri principali]---------------------------------------------
PWR_MGMT1    = 0x6B
PWR_MGMT2   = 0x6C
WHO_AM_I     = 0x75
SMPLRT_DIV   = 0x19
CONFIG       = 0x1A
GYRO_CONFIG  = 0x1B
ACCEL_CONFIG = 0x1C

# Blocco misure: burst-read da 0x3B per 14 bytes
ACCEL_XOUT_H = [0x3B]      # start register (must be list for tdwf.writeread)
SAD = 0x68

# Scale factors (default full-scale)
ACC_SCALE = 16384.0        # LSB/g  for ±2g (AFS_SEL=0)
GYR_SCALE = 131.0          # LSB/(deg/s) for ±250 dps (FS_SEL=0)

# -[Utility]-------------------------------------------------------------------
def twos_comp_16(val):
    """Convert unsigned 16-bit to signed 16-bit (two's complement)."""
    if val & 0x8000:
        val -= 0x10000
    return val

def parse_burst14(vals):
    """
    vals: list of 14 bytes from 0x3B..0x48
    returns: ax[g], ay[g], az[g], T[°C], gx[°/s], gy[°/s], gz[°/s]
    """
    # accel
    ax_raw = twos_comp_16((vals[0] << 8) | vals[1])
    ay_raw = twos_comp_16((vals[2] << 8) | vals[3])
    az_raw = twos_comp_16((vals[4] << 8) | vals[5])
    # temp
    t_raw  = twos_comp_16((vals[6] << 8) | vals[7])
    # gyro
    gx_raw = twos_comp_16((vals[8]  << 8) | vals[9])
    gy_raw = twos_comp_16((vals[10] << 8) | vals[11])
    gz_raw = twos_comp_16((vals[12] << 8) | vals[13])

    ax = ax_raw / ACC_SCALE
    ay = ay_raw / ACC_SCALE
    az = az_raw / ACC_SCALE

    gx = gx_raw / GYR_SCALE
    gy = gy_raw / GYR_SCALE
    gz = gz_raw / GYR_SCALE

    T  = t_raw / 340.0 + 36.53

    return ax, ay, az, T, gx, gy, gz

# -[Configurazione AD2]--------------------------------------------------------
ad2 = tdwf.AD2()
try:
    ad2.vdd = 3.3
    ad2.power(True)

    # Bus I2C
    i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
    devs = i2c.scan()
    for dev in devs:
        print(f"Device: 0x{dev:02x}")

    # Device I2C
    sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

    # -[Configurazione sensore]-------------------------------------------------
    # Reset (robusto): scrivi reset e attendi che il bit 7 torni 0
    sht.write([PWR_MGMT1, 0x80])
    time.sleep(0.01)
    t0 = time.time()
    while True:
        time.sleep(0.01)
        sht.writeread([PWR_MGMT1], 1)
        if (sht.vals[0] & 0x80) == 0:
            break
        if time.time() - t0 > 1.0:
            print("WARNING: reset timeout")
            break

    # Wake + enable sensors
    sht.write([PWR_MGMT1, 0x00])
    time.sleep(0.05)
    sht.write([PWR_MGMT2, 0x00])
    time.sleep(0.01)

    # Config filtro e sample rate (come nel tuo codice)
    sht.write([CONFIG, 0x03])
    time.sleep(0.01)
    sht.write([SMPLRT_DIV, 0x04])
    time.sleep(0.01)

    # Full-scale (default: accel ±2g, gyro ±250 dps)
    sht.write([GYRO_CONFIG, 0x00])
    time.sleep(0.01)
    sht.write([ACCEL_CONFIG, 0x00])
    time.sleep(0.01)

    # Check WHO_AM_I
    sht.writeread([WHO_AM_I], 1)
    print(f"WHO_AM_I: 0x{sht.vals[0]:02x}")

    # -[Gestione eventi]--------------------------------------------------------
    def on_close(event):
        global flag_run
        flag_run = False

    def on_key(event):
        global flag_run
        if event.key == 'escape':
            flag_run = False

    # -[Plot live (simile al template, leggero)]--------------------------------
    fig, ax = plt.subplots(figsize=(12,6))
    fig.canvas.mpl_connect("close_event", on_close)
    fig.canvas.mpl_connect("key_press_event", on_key)

    mytime = []
    TTv = []
    amag_v = []   # |a| in g
    wmag_v = []   # |w| in deg/s

    flag_first = True
    flag_run = True
    start_time = time.time()

    # -[Ciclo di misura]--------------------------------------------------------
    print("t[s], ax[g], ay[g], az[g], T[C], gx[deg/s], gy[deg/s], gz[deg/s]")
    while flag_run:
        time.sleep(0.1)  # periodicità stampa

        # Verifica comunicazione (opzionale ma utile in lab)
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("ALARM: Communication lost! Check wires.")
            continue

        # Burst-read 14 byte
        sht.writeread(ACCEL_XOUT_H, 14)
        vals = sht.vals[:14]
        ax_g, ay_g, az_g, T, gx, gy, gz = parse_burst14(vals)

        # Moduli per grafico (facoltativi)
        amag = math.sqrt(ax_g*ax_g + ay_g*ay_g + az_g*az_g)
        wmag = math.sqrt(gx*gx + gy*gy + gz*gz)

        t = time.time() - start_time
        mytime.append(t)
        TTv.append(T)
        amag_v.append(amag)
        wmag_v.append(wmag)

        # Output seriale/console (requisito Task 5)
        print(f"{t:7.2f}, {ax_g: .4f}, {ay_g: .4f}, {az_g: .4f}, {T: .2f}, {gx: .3f}, {gy: .3f}, {gz: .3f}")

        # Plot (stile simile al template)
        if flag_first:
            flag_first = False
            hpT,   = ax.plot(mytime, TTv,    "o", label="T [C]")
            hpA,   = ax.plot(mytime, amag_v, "o", label="|a| [g]")
            hpW,   = ax.plot(mytime, wmag_v, "o", label="|w| [deg/s]")
            ax.grid(True)
            ax.set_xlabel("Time [s]")
            ax.set_ylabel("Value")
            ax.set_title("MPU-6050 live (ESC to exit)")
            ax.legend()
            plt.show(block=False)
            plt.tight_layout()
        else:
            hpT.set_xdata(mytime); hpT.set_ydata(TTv)
            hpA.set_xdata(mytime); hpA.set_ydata(amag_v)
            hpW.set_xdata(mytime); hpW.set_ydata(wmag_v)
            ax.relim()
            ax.autoscale_view()
            fig.canvas.draw()
            fig.canvas.flush_events()

    plt.close(fig)

finally:
    print("\nCleaning up hardware...")
    try:
        plt.close('all')
    except:
        pass
    ad2.power(False)
    ad2.close()
    print("AD2 Power Off. Process safely terminated.")


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Device: 0x68
WHO_AM_I: 0x68
t[s], ax[g], ay[g], az[g], T[C], gx[deg/s], gy[deg/s], gz[deg/s]
   0.11, -0.9720,  0.0140,  0.0328,  37.14,  62.534, -160.244,  68.382
   0.30, -0.9707,  0.0143,  0.0348,  37.14,  136.794, -158.305, -1.969
   0.71, -0.9707,  0.0187,  0.0286,  37.14,  234.504, -164.153,  9.756
   0.88, -0.9733,  0.0148,  0.0303,  37.14, -226.687, -164.153,  21.481
   1.06, -0.9729,  0.0145,  0.0333,  37.14, -207.145, -152.427,  17.573
   1.22, -0.9738,  0.0170,  0.0303,  37.14, -179.786, -144.611,  25.389
   1.39, -0.9711,  0.0183,  0.0333,  37.14, -144.611, -164.153,  33.206
   1.55, -0.9718,  0.0155,  0.0289,  37.14, -171.969, -171.969,  44.931
   1.72, -0.9720,  0.0146,  0.0347,  37.14, -121.160, -203.237,  56.656
   1.90, -0.9723,  0.0125,  0.0322,  37.14, -105.527, -179.786,  56.656
   2.08, -0.9695,  0.0153,  0.0317,  37.14, -97.710, -187.603,  